## Work
1. 請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.regularizers import l1, l2, l1_l2

"""
建立神經網路，並加入 L1 或 L2
"""
def build_mlp_l1(input_shape, output_units=10, num_neurons=[512, 256, 128], l1_ratio=1e-4):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1), 
                                   kernel_regularizer=l1(l1_ratio))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1),
                                   kernel_regularizer=l1(l1_ratio))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model
def build_mlp_l1_l2(input_shape, output_units=10, num_neurons=[512, 256, 128], l1_l2_ratio=1e-4):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1), 
                                   kernel_regularizer=l1_l2(l1_l2_ratio))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1),
                                   kernel_regularizer=l1_l2(l1_12_ratio))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95
L2_EXP = [1e-2, 1e-4, 1e-8, 1e-12]

In [7]:
results_l1 = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
results_l1_l2= {}
"""
使用迴圈建立不同的帶不同 L1/L2 的模型並訓練
"""
for regulizer_ratio in L2_EXP:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with Regulizer = %.6f" % (regulizer_ratio))
    model = build_mlp_l1(input_shape=x_train.shape[1:], l1_ratio=regulizer_ratio)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-l2-%s" % str(regulizer_ratio)
    results_l1[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}
for regulizer_ratio in L2_EXP:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with Regulizer = %.6f" % (regulizer_ratio))
    model = build_mlp_l1_l2(input_shape=x_train.shape[1:], l1_l2_ratio=regulizer_ratio)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-l2-%s" % str(regulizer_ratio)
    results_l1_l2[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}


Experiment with Regulizer = 0.010000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [=====================

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 24s 486us/step - loss: 5.9938 - acc: 0.2808 - val_loss: 5.8053 - val_acc: 0.3465
Epoch 2/50
50000/50000 [==============================] - 23s 453us/step - loss: 5.7216 - acc: 0.3639 - val_loss: 5.6484 - val_acc: 0.3757
Epoch 3/50
50000/50000 [==============================] - 23s 456us/step - loss: 5.5803 - acc: 0.3915 - val_loss: 5.5169 - val_acc: 0.4015
Epoch 4/50
50000/50000 [==============================] - 23s 457us/step - loss: 5.4574 - acc: 0.4113 - val_loss: 5.4072 - val_acc: 0.4143
Epoch 5/50
50000/50000 [==============================] - 22s 449us/step - loss: 5.3471 - acc: 0.4269 - val_loss: 5.2903 - val_acc: 0.4344
Epoch 6/50
50000/50000 [==============================] - 22s 449us/step - loss: 5.2425 - acc: 0.4431 - val_loss: 5.1968 - val_acc: 0.4490
Epoch 7/50
50000/50000 [==============================] - 23s 458us/step - loss: 5.1459 - acc: 0.4545 - val_loss: 5.

50000/50000 [==============================] - 21s 425us/step - loss: 2.0236 - acc: 0.2778 - val_loss: 1.8637 - val_acc: 0.3450
Epoch 2/50
50000/50000 [==============================] - 20s 407us/step - loss: 1.8006 - acc: 0.3709 - val_loss: 1.7524 - val_acc: 0.3893
Epoch 3/50
50000/50000 [==============================] - 20s 402us/step - loss: 1.7181 - acc: 0.3992 - val_loss: 1.6880 - val_acc: 0.4063
Epoch 4/50
50000/50000 [==============================] - 20s 403us/step - loss: 1.6611 - acc: 0.4201 - val_loss: 1.6437 - val_acc: 0.4231
Epoch 5/50
50000/50000 [==============================] - 20s 403us/step - loss: 1.6116 - acc: 0.4356 - val_loss: 1.6059 - val_acc: 0.4376
Epoch 6/50
50000/50000 [==============================] - 21s 411us/step - loss: 1.5728 - acc: 0.4498 - val_loss: 1.5608 - val_acc: 0.4505
Epoch 7/50
50000/50000 [==============================] - 20s 402us/step - loss: 1.5359 - acc: 0.4623 - val_loss: 1.5385 - val_acc: 0.4567
Epoch 8/50
50000/50000 [==============

50000/50000 [==============================] - 23s 451us/step - loss: 2.0374 - acc: 0.2767 - val_loss: 1.8701 - val_acc: 0.3366
Epoch 2/50
50000/50000 [==============================] - 21s 416us/step - loss: 1.8063 - acc: 0.3639 - val_loss: 1.7594 - val_acc: 0.3741
Epoch 3/50
50000/50000 [==============================] - 21s 413us/step - loss: 1.7220 - acc: 0.3949 - val_loss: 1.6902 - val_acc: 0.4030
Epoch 4/50
50000/50000 [==============================] - 21s 418us/step - loss: 1.6658 - acc: 0.4158 - val_loss: 1.6426 - val_acc: 0.4244
Epoch 5/50
50000/50000 [==============================] - 21s 421us/step - loss: 1.6173 - acc: 0.4350 - val_loss: 1.5958 - val_acc: 0.4373
Epoch 6/50
50000/50000 [==============================] - 21s 430us/step - loss: 1.5816 - acc: 0.4448 - val_loss: 1.5690 - val_acc: 0.4458
Epoch 7/50
50000/50000 [==============================] - 23s 454us/step - loss: 1.5474 - acc: 0.4567 - val_loss: 1.5548 - val_acc: 0.4497
Epoch 8/50
50000/50000 [==============

NameError: name 'l1_12_ratio' is not defined

In [ ]:

"""Code Here
將結果繪出
"""
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results_l1.keys()):
    plt.plot(range(len(results_l1[cond]['train-loss'])),results_l1[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results_l1[cond]['valid-loss'])),results_l1[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results_l1.keys()):
    plt.plot(range(len(results_l1[cond]['train-acc'])),results_l1[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results_l1[cond]['valid-acc'])),results_l1[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()

In [ ]:
"""Code Here
將結果繪出
"""
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results_l1_l2.keys()):
    plt.plot(range(len(results_l1_l2[cond]['train-loss'])),results_l1_l2[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results_l1_l2[cond]['valid-loss'])),results_l1_l2[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results_l1_l2.keys()):
    plt.plot(range(len(results_l1_l2[cond]['train-acc'])),results_l1_l2[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results_l1_l2[cond]['valid-acc'])),results_l1_l2[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()